In [2]:
import requests
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Image
# from statsmodels.stats.outliers_influence import variance_inflation_factor
sns.set_style('whitegrid')
%matplotlib inline

# import plotly.express as px

from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import LeaveOneOut
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder


from matplotlib import pyplot


In [3]:



# puuid 
puuid = "CUcPTA2mHGm2O_wFyZl0p5pC4s2OV-uXAt7cnyuK_ylHZNWuHji609Zv6dyvBKU4BuKJ1S1tIru4GA"


# Chỉ số trận đấu đầu tiên và cuối cùng trong danh sách
begin_index = 0
end_index = 100

# API key của bạn
api_key = "RGAPI-2172e0e8-babe-4c52-be09-fb3071a691de"

# Tạo đường dẫn truy vấn API lấy danh sách match id
requestUrl_for_matchIds = f"https://sea.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?type=ranked&start={begin_index}&count={end_index}&api_key={api_key}"

# Gửi truy vấn API và phân tích phản hồi JSON
responseMatchId = requests.get(requestUrl_for_matchIds)
matchIds = json.loads(responseMatchId.text)
# print(matchIds)
listInfo = []


keys_to_delete = ["challenges","perks","puuid","summonerId","riotIdName","riotIdTagline",]

# Gửi truy vấn API lấy thông tin của từng match
for matchId in matchIds :
    requestUrl_for_matchInfor_byId = f"https://sea.api.riotgames.com/lol/match/v5/matches/{matchId}?api_key={api_key}"
    respondMatch = requests.get(requestUrl_for_matchInfor_byId)
    data = json.loads(respondMatch.text)
    # print(data.keys())
    if "info" in data: 
        dataPar = data["info"]["participants"] 
        # print(len(dataPar))
        for par in dataPar:
            # print(len(par))
            par_copy = par.copy()  # Tạo một bản sao độc lập của par
            for key in keys_to_delete:
                par_copy.pop(key, None)
            listInfo.append(par_copy)
df = pd.DataFrame(listInfo)


In [4]:
df.shape

(990, 117)

In [5]:
df.head()

,allInPings,assistMePings,assists,baitPings,baronKills,basicPings,bountyLevel,champExperience,champLevel,championId,...,turretKills,turretTakedowns,turretsLost,unrealKills,visionClearedPings,visionScore,visionWardsBoughtInGame,wardsKilled,wardsPlaced,win
0,3,10,15,2,0,0,0,25359,18,126,...,1,4,7,0,0,44,2,9,7,False
1,1,0,11,0,1,0,0,25631,18,245,...,0,1,7,0,0,36,1,11,3,False
2,0,2,11,0,0,0,0,21040,18,246,...,1,1,7,0,0,30,3,9,4,False
3,1,1,24,0,1,0,1,26383,18,222,...,5,7,7,0,0,57,6,5,18,False
4,0,20,27,1,0,0,0,18343,17,117,...,2,6,7,0,0,144,4,15,62,False


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 990 entries, 0 to 989
Columns: 117 entries, allInPings to win
dtypes: bool(9), float64(2), int64(100), object(6)
memory usage: 844.1+ KB


In [7]:
X = df.iloc[:,:-1]
y = df.iloc[:,-1]


In [8]:
# Kiểm tra các cột có dữ liệu trống cho từng kiểu dữ liệu
columns_with_missing_values_bool = X.select_dtypes(include=bool).columns[X.select_dtypes(include=bool).isna().any()]
columns_with_missing_values_float = X.select_dtypes(include=float).columns[X.select_dtypes(include=float).isna().any()]
columns_with_missing_values_int = X.select_dtypes(include=int).columns[X.select_dtypes(include=int).isna().any()]
columns_with_missing_values_object = X.select_dtypes(include=object).columns[X.select_dtypes(include=object).isna().any()]

# In ra các cột có dữ liệu trống cho từng kiểu dữ liệu
print("Các cột bool có dữ liệu trống:", columns_with_missing_values_bool)
print("Các cột float có dữ liệu trống:", columns_with_missing_values_float)
print("Các cột int có dữ liệu trống:", columns_with_missing_values_int)
print("Các cột object có dữ liệu trống:", columns_with_missing_values_object)

Các cột bool có dữ liệu trống: Index([], dtype='object')
Các cột float có dữ liệu trống: Index(['totalAllyJungleMinionsKilled', 'totalEnemyJungleMinionsKilled'], dtype='object')
Các cột int có dữ liệu trống: Index([], dtype='object')
Các cột object có dữ liệu trống: Index([], dtype='object')


In [10]:
def delete_unique_columns(df) :
    for column in df.columns:
        unique_values = df[column].unique()
        if len(unique_values) == 1:
            print(f"Cột {column} chỉ có giá trị duy nhất: {unique_values[0]}. Sẽ bị loại bỏ.")
            df.drop(column, axis=1, inplace=True)
delete_unique_columns(X)


Cột basicPings chỉ có giá trị duy nhất: 0. Sẽ bị loại bỏ.
Cột sightWardsBoughtInGame chỉ có giá trị duy nhất: 0. Sẽ bị loại bỏ.
Cột unrealKills chỉ có giá trị duy nhất: 0. Sẽ bị loại bỏ.


In [9]:
object_columns = X.select_dtypes(include=['object'])
object_columns

,championName,individualPosition,lane,role,summonerName,teamPosition
0,Jayce,TOP,TOP,SOLO,u make me suyy,TOP
1,Ekko,JUNGLE,JUNGLE,NONE,Evill øƒ Haki,JUNGLE
2,Qiyana,MIDDLE,MIDDLE,SOLO,Thiên Khung,MIDDLE
3,Jinx,BOTTOM,BOTTOM,CARRY,const Nghiện,BOTTOM
4,Lulu,UTILITY,BOTTOM,SUPPORT,H 28 04 24,UTILITY
...,...,...,...,...,...,...
985,Jax,TOP,TOP,SOLO,SeraChan,TOP
986,Graves,JUNGLE,JUNGLE,NONE,const Nghiện,JUNGLE
987,Ryze,MIDDLE,MIDDLE,SOLO,SFC HunterH,MIDDLE
988,Jinx,BOTTOM,BOTTOM,CARRY,Đào Tuệ Linh,BOTTOM


In [11]:

def convert_cat(df, col) :
    df[col] = df[col].astype("category")
convert_cat(X,"teamPosition")
X['teamPosition']

0          TOP
1       JUNGLE
2       MIDDLE
3       BOTTOM
4      UTILITY
        ...   
985        TOP
986     JUNGLE
987     MIDDLE
988     BOTTOM
989    UTILITY
Name: teamPosition, Length: 990, dtype: category
Categories (5, object): ['BOTTOM', 'JUNGLE', 'MIDDLE', 'TOP', 'UTILITY']

In [12]:
columns_to_drop = ['championName','individualPosition','lane','role','summonerName','totalAllyJungleMinionsKilled','totalEnemyJungleMinionsKilled']
X.drop(columns=columns_to_drop, axis=1, inplace=True)
X.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 990 entries, 0 to 989
Columns: 106 entries, allInPings to wardsPlaced
dtypes: bool(8), category(1), int64(97)
memory usage: 759.3 KB


In [13]:
label_encoder = LabelEncoder()

# Mã hóa nhãn
X['teamPosition'] = label_encoder.fit_transform(X['teamPosition'])


In [14]:
correlation_matrix = X.corr()

# Kiểm tra tương quan giữa các cột
threshold = 0.8  # Ngưỡng tương quan
correlated_columns = set()
for i in range(len(correlation_matrix.columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) > threshold:
            col_i = correlation_matrix.columns[i]
            col_j = correlation_matrix.columns[j]
            correlated_columns.add(col_i)
            correlated_columns.add(col_j)
            print(f"Cột {col_i} và cột {col_j} có tương quan cao. Sẽ giữ lại một trong hai cột.")

# Lược bỏ các cột có tương quan cao
X.drop(correlated_columns, axis=1, inplace=True)

Cột champLevel và cột champExperience có tương quan cao. Sẽ giữ lại một trong hai cột.
Cột damageDealtToTurrets và cột damageDealtToBuildings có tương quan cao. Sẽ giữ lại một trong hai cột.
Cột gameEndedInEarlySurrender và cột eligibleForProgression có tương quan cao. Sẽ giữ lại một trong hai cột.
Cột goldEarned và cột champExperience có tương quan cao. Sẽ giữ lại một trong hai cột.
Cột goldEarned và cột champLevel có tương quan cao. Sẽ giữ lại một trong hai cột.
Cột goldSpent và cột champExperience có tương quan cao. Sẽ giữ lại một trong hai cột.
Cột goldSpent và cột champLevel có tương quan cao. Sẽ giữ lại một trong hai cột.
Cột goldSpent và cột goldEarned có tương quan cao. Sẽ giữ lại một trong hai cột.
Cột kills và cột killingSprees có tương quan cao. Sẽ giữ lại một trong hai cột.
Cột magicDamageDealtToChampions và cột magicDamageDealt có tương quan cao. Sẽ giữ lại một trong hai cột.
Cột physicalDamageDealtToChampions và cột physicalDamageDealt có tương quan cao. Sẽ giữ lại một tr

In [16]:
filtered_df = X.filter(regex='^(?!.*Pings).*$')
filtered_df

,assists,baronKills,bountyLevel,championId,championTransform,consumablesPurchased,damageDealtToObjectives,doubleKills,dragonKills,firstBloodAssist,...,totalHealsOnTeammates,totalMinionsKilled,totalTimeCCDealt,totalUnitsHealed,tripleKills,trueDamageDealt,trueDamageDealtToChampions,trueDamageTaken,turretTakedowns,wardsKilled
0,15,0,0,126,0,8,16086,2,0,False,...,0,228,467,1,1,7191,4896,2402,4,9
1,11,1,0,245,0,6,51350,1,3,True,...,0,79,615,1,0,58099,5766,3238,1,11
2,11,0,0,246,0,5,6281,3,0,False,...,0,196,124,1,0,1667,797,4613,1,9
3,24,1,1,222,0,13,52743,1,1,False,...,0,332,184,1,0,41298,5149,2270,7,5
4,27,0,0,117,0,9,3543,0,0,False,...,2779,60,244,7,0,7795,2092,1084,6,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
985,5,0,0,24,0,5,3112,1,0,False,...,0,103,107,1,0,7846,956,701,2,4
986,8,0,0,104,0,6,24009,0,1,True,...,0,59,634,1,0,28351,739,505,1,7
987,3,0,0,13,0,2,3664,0,0,False,...,0,169,95,1,0,226,60,800,2,3
988,3,0,0,222,0,11,0,0,0,False,...,505,130,72,3,0,2579,339,403,0,2
